In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = './train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
np.shape(tX)
#np.shape(y)

(250000, 30)

In [4]:
#check for any NaN values
nans = np.isnan(tX)
np.unique(nans)

array([False])

## Feature reduction

### Removing features that are mostly one value

In [5]:
def feature_selection(tX,val=-999):
    #there is a lot of -999 values in the data
    #count the occurences for each feature
    f_to_remove = []
    for f in range(np.shape(tX)[1]):
        #print(tX[:,f])
        unique, counts = np.unique(tX[:,f], return_counts=True)
        count_dic = dict(zip(unique, counts))
        if val in count_dic:
            nb_999 = count_dic[val]
            nb_tot = np.shape(tX)[0]
            print(f"feature: ",f)
            per = (nb_999/nb_tot)*100
            print(f"number of ", val," values",nb_999,"which is",per,"%")
            if per > 10:
                f_to_remove.append(f)
    print(f"to remove",f_to_remove)
    return f_to_remove

In [8]:
 f_to_remove

[0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28]

In [7]:
#there is a lot of -999 values in the data
#count the occurences for each feature
f_to_remove = []
for f in range(np.shape(tX)[1]):
    #print(tX[:,f])
    unique, counts = np.unique(tX[:,f], return_counts=True)
    count_dic = dict(zip(unique, counts))
    if -999 in count_dic:
        nb_999 = count_dic[-999]
        nb_tot = np.shape(tX)[0]
        print(f"feature: ",f)
        per = (nb_999/nb_tot)*100
        print(f"number of -999 values",nb_999,"which is",per,"%")
        if per > 10:
            f_to_remove.append(f)
print(f"to remove",f_to_remove)

feature:  0
number of -999 values 38114 which is 15.245600000000001 %
feature:  4
number of -999 values 177457 which is 70.9828 %
feature:  5
number of -999 values 177457 which is 70.9828 %
feature:  6
number of -999 values 177457 which is 70.9828 %
feature:  12
number of -999 values 177457 which is 70.9828 %
feature:  23
number of -999 values 99913 which is 39.9652 %
feature:  24
number of -999 values 99913 which is 39.9652 %
feature:  25
number of -999 values 99913 which is 39.9652 %
feature:  26
number of -999 values 177457 which is 70.9828 %
feature:  27
number of -999 values 177457 which is 70.9828 %
feature:  28
number of -999 values 177457 which is 70.9828 %
to remove [0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28]


In [8]:
#delete the selected features
tX_red_f = np.delete(tX,f_to_remove,1)
np.shape(tX_red_f)

(250000, 19)

### Checking feature similarity

In [9]:
np.shape(tX_red_f)

(250000, 19)

In [10]:
def f_corr(corr_thr):
    #find correlation between the features
    #corr_thr = 8e-1
    corr = np.absolute(np.corrcoef(tX_red_f.T))
    bool_cor = np.greater(corr,corr_thr)
    f1,f2 = np.where(bool_cor)
    for i in range(len(f1)):
        if ~(f1[i] == f2[i]):
            corr_val = corr[f1[i],f2[i]]
            print(f"Features ",f1[i],"and",f2[i],"are strongly correlated. Value:",corr_val)

In [11]:
#find correlation between the features
corr_thr = 8e-1
corr = np.absolute(np.corrcoef(tX_red_f.T))
bool_cor = np.greater(corr,corr_thr)
f1,f2 = np.where(bool_cor)
#print(f1,f2)
for i in range(len(f1)):
    if ~(f1[i] == f2[i]):
        corr_val = corr[f1[i],f2[i]]
        print(f"Features ",f1[i],"and",f2[i],"are strongly correlated. Value:",corr_val)

Features  2 and 5 are strongly correlated. Value: 0.8327330666331234
Features  2 and 18 are strongly correlated. Value: 0.8086162641281681
Features  5 and 2 are strongly correlated. Value: 0.8327330666331234
Features  5 and 16 are strongly correlated. Value: 0.9044814595684981
Features  5 and 18 are strongly correlated. Value: 0.9656283889163993
Features  16 and 5 are strongly correlated. Value: 0.9044814595684981
Features  16 and 18 are strongly correlated. Value: 0.8844128574100243
Features  17 and 18 are strongly correlated. Value: 0.8043255943442028
Features  18 and 2 are strongly correlated. Value: 0.8086162641281681
Features  18 and 5 are strongly correlated. Value: 0.9656283889163995
Features  18 and 16 are strongly correlated. Value: 0.8844128574100243
Features  18 and 17 are strongly correlated. Value: 0.8043255943442027


Very strong corr between features 5,16,18 **The three features can be replaced by just feature 5**

additional corr between features 2 and 5,18 **2 can also be removed to just keep 5**

In [12]:
f_to_remove = np.array([2,16,18])

In [13]:
#delete the selected features
tX = np.delete(tX_red_f,f_to_remove,1)
np.shape(tX)

(250000, 16)

## Next step

In [49]:
N = np.shape(tX)[0]
np.arange(N)
choice = np.random.choice(N,size=1000)
new_tX = tX[choice, :]
new_y = y[choice]

## Running ML

In [50]:
y, tX, ids

(array([ 1., -1., -1., ...,  1., -1., -1.]),
 array([[ 51.655,  97.827,   3.064, ...,  16.824,  -0.277,   2.   ],
        [ 68.768, 103.235,   3.473, ...,  44.704,  -1.916,   1.   ],
        [162.172, 125.953,   3.148, ...,  54.283,  -2.186,   1.   ],
        ...,
        [ 60.526,  75.839,   2.39 , ...,  23.419,  -2.89 ,   1.   ],
        [ 19.362,  68.812,   3.365, ...,  12.15 ,   0.811,   0.   ],
        [ 72.756,  70.831,   2.025, ...,  40.729,  -1.596,   0.   ]]),
 array([100000, 100001, 100002, ..., 349997, 349998, 349999]))

In [67]:
np.shape(new_y)

(1000,)

In [51]:
def sigmoid(t):
    """apply the sigmoid function on t."""
    return 1.0 / (1+np.exp(-t))

In [52]:
def calculate_loss(y, tx, w):
    """compute the loss: negative log likelihood."""
    sigma = sigmoid(tx.dot(w))
    loss = (y.T.dot(np.log(sigma)) + (1-y).T.dot(np.log(1-sigma)))
    return np.squeeze(-1*loss)

In [53]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    sigma = sigmoid(tx.dot(w))
    return tx.T.dot(sigma - y)

In [61]:
def calculate_hessian(y, tx, w):
    """return the Hessian of the loss function."""
    sigma = sigmoid(tx.dot(w))
    S = np.multiply(sigma,(1-sigma))
    S = np.diag(S.T[0])
    return tx.T.dot(S).dot(tx)
    raise NotImplementedError

In [62]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient"""
    loss = calculate_loss(y,tx,w) + lambda_*w.T.dot(w)
    grad = calculate_gradient(y,tx,w) + 2*lambda_*w
    hess = calculate_hessian(y,tx,w)
    return loss, grad, hess
    raise NotImplementedError

In [63]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, grad, hess = penalized_logistic_regression(y, tx, w, lambda_)
    w = w - gamma * grad
    return loss, w

In [68]:
def logistic_regression_penalized_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 10000
    gamma = 0.01
    lambda_ = 0.1
    threshold = 1e-8
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_penalized_gradient_descent",True)
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    
logistic_regression_penalized_gradient_descent_demo(new_y, new_tX)

Current iteration=0, loss=[[693.14718056]]


/tmp/ipykernel_457/1417711890.py:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1+np.exp(-t))
/tmp/ipykernel_457/1390517956.py:4: RuntimeWarning: divide by zero encountered in log
  loss = (y.T.dot(np.log(sigma)) + (1-y).T.dot(np.log(1-sigma)))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Group data


# Imputation Using Zero/Constant Values
tx = np.where(tX==-999, 0, tX) 

In [ ]:
# Linear regression using gradient descent
from implementations import *

# Define the parameters of the algorithm.
max_iters = 50
gamma = 3 # use linspace to test various gamma values and look for the best ?

# Initialization
initial_w = np.zeros(tx.shape[1]+1)
tx_offset = np.empty([tx.shape[0], tx.shape[1]+1])
tx_offset[:, 0] = np.ones([tx.shape[0]]) 
tx_offset[:, 1:] = tx
loss_GD, w_GD = least_squares_GD(y, tx_offset, initial_w, max_iters, gamma)

print("Gradient Descent: loss={l}, w = {w}".format(
    l=loss_GD, w=w_GD))


In [ ]:
# Linear regression using stochastic gradient descent

# Define the parameters of the algorithm.
max_iters = 50
gamma = 3 # use linspace to test various gamma values and look for the best ?

# Initialization
# initial_w = np.zeros(tx.shape[1]+1)
# tx_offset = np.empty([tx.shape[0], tx.shape[1]+1])
# tx_offset[:, 0] = np.ones([tx.shape[0]]) 
# tx_offset[:, 1:] = tx
loss_GD, w_GD = least_squares_SGD(y, tx_offset, initial_w, max_iters, gamma)

print("Stochastic Gradient Descent: loss={l}, w = {w}".format(
    l=loss_GD, w=w_GD))

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)